# Generatiiviset verkot

Toistuvat neuroverkot (Recurrent Neural Networks, RNN) ja niiden portilliset solumuunnelmat, kuten Long Short Term Memory Cells (LSTM) ja Gated Recurrent Units (GRU), tarjoavat mekanismin kielen mallintamiseen, eli ne voivat oppia sanojen järjestyksen ja ennustaa seuraavan sanan sekvenssissä. Tämä mahdollistaa RNN:ien käytön **generatiivisissa tehtävissä**, kuten tavallisessa tekstin generoinnissa, konekäännöksessä ja jopa kuvatekstien luomisessa.

RNN-arkkitehtuurissa, jota käsittelimme edellisessä osiossa, jokainen RNN-yksikkö tuotti seuraavan piilotilan ulostulona. Voimme kuitenkin lisätä jokaiselle toistuvalle yksikölle toisen ulostulon, mikä mahdollistaa **sekvenssin** tuottamisen (joka on yhtä pitkä kuin alkuperäinen sekvenssi). Lisäksi voimme käyttää RNN-yksiköitä, jotka eivät ota syötettä jokaisessa vaiheessa, vaan ainoastaan jonkin alkuperäisen tilavektorin, ja tuottavat sitten ulostulosekvenssin.

Tässä muistikirjassa keskitymme yksinkertaisiin generatiivisiin malleihin, jotka auttavat meitä tuottamaan tekstiä. Yksinkertaisuuden vuoksi rakennetaan **merkki-tason verkko**, joka generoi tekstiä kirjain kerrallaan. Koulutuksen aikana meidän täytyy ottaa jokin tekstikorpus ja jakaa se kirjainsekvensseihin.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Rakennetaan merkkisanasto

Merkkitason generatiivisen verkon rakentamiseksi meidän täytyy jakaa teksti yksittäisiin merkkeihin sanojen sijaan. Tämä voidaan tehdä määrittelemällä erilainen tokenisaattori:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Katsotaan esimerkki siitä, miten voimme koodata tekstin aineistostamme:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Generatiivisen RNN:n kouluttaminen

Tapa, jolla koulutamme RNN:n tuottamaan tekstiä, on seuraava. Jokaisella askeleella otamme `nchars` merkin pituisen merkkijonon ja pyydämme verkkoa tuottamaan seuraavan ulostulomerkin jokaiselle syötemerkille:

![Kuva, joka näyttää esimerkin RNN:n tuottamasta sanasta 'HELLO'.](../../../../../translated_images/fi/rnn-generate.56c54afb52f9781d.webp)

Riippuen todellisesta tilanteesta, saatamme haluta sisällyttää myös joitakin erikoismerkkejä, kuten *sekvenssin loppu* `<eos>`. Meidän tapauksessamme haluamme vain kouluttaa verkon loputtomaan tekstin tuottamiseen, joten kiinnitämme jokaisen sekvenssin koon `nchars`-merkkien pituiseksi. Näin ollen jokainen koulutusesimerkki koostuu `nchars` syötteistä ja `nchars` ulostuloista (jotka ovat syötesekvenssi siirrettynä yhden symbolin verran vasemmalle). Minibatch koostuu useista tällaisista sekvensseistä.

Tapa, jolla luomme minibatchit, on ottaa jokainen uutisteksti, jonka pituus on `l`, ja luoda siitä kaikki mahdolliset syöte-ulostulo-yhdistelmät (näitä yhdistelmiä on `l-nchars`). Ne muodostavat yhden minibatchin, ja minibatchien koko vaihtelee jokaisella koulutusaskeleella.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Nyt määritellään generaattoriverkko. Se voi perustua mihin tahansa rekursiiviseen soluun, joita käsiteltiin edellisessä osiossa (yksinkertainen, LSTM tai GRU). Esimerkissämme käytämme LSTM:ää.

Koska verkko ottaa syötteenä merkkejä ja sanaston koko on melko pieni, emme tarvitse upotuskerrosta; yksi-hot-koodattu syöte voi mennä suoraan LSTM-soluun. Kuitenkin, koska syötteenä annetaan merkkien numerot, ne täytyy yksi-hot-koodata ennen kuin ne välitetään LSTM:lle. Tämä tehdään kutsumalla `one_hot`-funktiota `forward`-kulkuprosessin aikana. Ulostulon kooderi olisi lineaarinen kerros, joka muuntaa piilotilan yksi-hot-koodatuksi ulostuloksi.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

Harjoittelun aikana haluamme pystyä näytteistämään generoituja tekstejä. Tätä varten määrittelemme `generate`-funktion, joka tuottaa merkkijonon, jonka pituus on `size`, alkaen alkuperäisestä merkkijonosta `start`.

Näin se toimii. Ensin syötämme koko `start`-merkkijonon verkkoon ja otamme ulostulotilan `s` sekä seuraavan ennustetun merkin `out`. Koska `out` on one-hot-koodattu, käytämme `argmax`-funktiota saadaksemme merkin `nc` indeksin sanastossa, ja käytämme `itos`-funktiota selvittääksemme todellisen merkin, jonka lisäämme tuloksena olevaan merkkilistaan `chars`. Tämä yhden merkin generointiprosessi toistetaan `size` kertaa, jotta saadaan tarvittava määrä merkkejä.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Aloitetaan harjoittelu! Harjoittelusilmukka on lähes sama kuin kaikissa aiemmissa esimerkeissämme, mutta tarkkuuden sijaan tulostamme satunnaisesti generoituja tekstejä joka 1000. epookin kohdalla.

Erityistä huomiota on kiinnitettävä siihen, miten laskemme häviön. Meidän täytyy laskea häviö annetun one-hot-koodatun tulosteen `out` ja odotetun tekstin `text_out` perusteella, joka on merkkien indeksien lista. Onneksi `cross_entropy`-funktio odottaa ensimmäisenä argumenttina normalisoimatonta verkon tulostetta ja toisena argumenttina luokan numeroa, mikä on juuri se, mitä meillä on. Se myös suorittaa automaattisen keskiarvon laskemisen minibatchin koon perusteella.

Rajoitamme harjoittelun myös `samples_to_train`-näytteisiin, jotta emme joudu odottamaan liian kauan. Kannustamme sinua kokeilemaan ja testaamaan pidempää harjoittelua, mahdollisesti useiden epookkien ajan (tässä tapauksessa sinun täytyisi luoda toinen silmukka tämän koodin ympärille).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

Tämä esimerkki tuottaa jo varsin hyvää tekstiä, mutta sitä voidaan parantaa useilla tavoilla:

* **Parempi minibatchien luonti**. Tapa, jolla valmistelimme dataa koulutusta varten, oli luoda yksi minibatch yhdestä näytteestä. Tämä ei ole ihanteellista, koska minibatchit ovat kaikki erikokoisia, ja joitakin niistä ei voida edes luoda, koska teksti on pienempi kuin `nchars`. Lisäksi pienet minibatchit eivät kuormita GPU:ta riittävästi. Olisi viisaampaa ottaa yksi suuri tekstikappale kaikista näytteistä, luoda sitten kaikki syöte-tulos-parit, sekoittaa ne ja luoda samankokoisia minibatcheja.

* **Monikerroksinen LSTM**. On järkevää kokeilla 2 tai 3 kerrosta LSTM-soluja. Kuten mainitsimme edellisessä osiossa, jokainen LSTM-kerros poimii tiettyjä kuvioita tekstistä, ja merkkitason generaattorin tapauksessa voimme odottaa alemman LSTM-tason vastaavan tavujen poimimisesta ja ylempien tasojen - sanoista ja sanayhdistelmistä. Tämä voidaan toteuttaa yksinkertaisesti antamalla kerrosten lukumäärä -parametri LSTM-rakentajalle.

* Voit myös haluta kokeilla **GRU-yksiköitä** ja nähdä, mitkä toimivat paremmin, sekä **erilaisia piilokerroksen kokoja**. Liian suuri piilokerros voi johtaa ylisovittamiseen (esim. verkko oppii tarkan tekstin), ja pienempi koko ei välttämättä tuota hyvää tulosta.


## Pehmeä tekstin generointi ja lämpötila

Aiemmassa `generate`-määritelmässä valitsimme aina seuraavaksi merkiksi sen, jolla oli korkein todennäköisyys. Tämä johti usein siihen, että teksti "kiersi" samoja merkkijonoja uudelleen ja uudelleen, kuten tässä esimerkissä:
```
today of the second the company and a second the company ...
```

Jos kuitenkin tarkastelemme seuraavan merkin todennäköisyysjakaumaa, voi olla, että muutaman korkeimman todennäköisyyden ero ei ole suuri, esimerkiksi yksi merkki voi olla todennäköisyydellä 0.2 ja toinen 0.19, jne. Esimerkiksi, kun etsitään seuraavaa merkkiä jaksossa '*play*', seuraava merkki voi yhtä hyvin olla joko välilyönti tai **e** (kuten sanassa *player*).

Tämä johtaa siihen päätelmään, että ei ole aina "reilua" valita merkkiä, jolla on korkein todennäköisyys, koska toisen korkein todennäköisyys voi silti johtaa merkitykselliseen tekstiin. Viisaampaa on **näytteistää** merkkejä verkon tuottaman todennäköisyysjakauman perusteella.

Tämä näytteistäminen voidaan tehdä `multinomial`-funktiolla, joka toteuttaa niin kutsutun **multinomiaalisen jakauman**. Alla on määritelty funktio, joka toteuttaa tämän **pehmeän** tekstin generoinnin:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

Olemme ottaneet käyttöön yhden lisäparametrin nimeltä **lämpötila**, jota käytetään osoittamaan, kuinka tiukasti meidän tulisi pitäytyä korkeimmassa todennäköisyydessä. Jos lämpötila on 1.0, teemme reilua multinomiaalisia otantoja, ja kun lämpötila kasvaa äärettömyyteen - kaikki todennäköisyydet muuttuvat yhtä suuriksi, ja valitsemme seuraavan merkin satunnaisesti. Alla olevassa esimerkissä voimme havaita, että teksti muuttuu merkityksettömäksi, kun nostamme lämpötilaa liikaa, ja se muistuttaa "kierrätettyä" tiukasti generoituvaa tekstiä, kun lämpötila lähestyy arvoa 0.



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
